In [161]:
from numpy.core.arrayprint import printoptions
from numpy.lib.function_base import extract
import scipy.io
import numpy as np
import pandas as pd
import os
import json

In [207]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)
        
def splitcsvjson4(x):
    js = " "
    df = pd.DataFrame(x['X'][0])
    js = x.to_dict()
    del js['X']
    for i,j in js.items():
        if(not(isinstance(j[0][0],str))  and (i != 'channels' and i != 'classes' and i != 'classes_stim')):
            js[i] = list(extract_list(j)[0])
        elif (isinstance(j[0][0],str)):
            js[i] = j[0][0]
        else:
            js[i] = [list(k)[0] for k in j[0][0]]
    js = json.dumps(js, indent=4, cls=NpEncoder)
    return df, js

def read_matfile4(filename):
    data = scipy.io.loadmat(filename)['data']
    cols=[]
    for i in data.dtype.fields.items():
        if '__' not in i :
            cols.append(i)
    df=pd.DataFrame()
    for i in [x[0] for x in cols]:
        if '__' not in i :
            df[i]=(data[i]).ravel()
    return df

def convert_mat4(file):
    df = read_matfile4(file)
    direction = file.split('.')[0]
#     os.mkdir(direction)
    df_1, metadata = splitcsvjson4(df)
    df_1 = df_1.set_index(0)
    df_1.to_csv(direction+'/data.csv')
    with open(direction+"/metadata.json","w") as f:
        f.write(metadata)

In [208]:
convert_mat4('A01.mat')

In [170]:
with open("example.json","w") as f:
        f.write(metadata)